In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

load_dotenv()
TRAIN_SET = os.getenv("TRAIN_PATH")
TEST_SET  = os.getenv("TEST_PATH")

train = pd.read_csv(TRAIN_SET)
test = pd.read_csv(TEST_SET) 

train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Pulizia dei dati: il test set presenta 59 colonne. Il train set ne presenta 82. Rimuovo le colonne in eccesso 

In [3]:
test_features_serie = pd.Series(test.columns)
print(test_features_serie)

0                                         id
1                  Basic_Demos-Enroll_Season
2                            Basic_Demos-Age
3                            Basic_Demos-Sex
4                                CGAS-Season
5                            CGAS-CGAS_Score
6                            Physical-Season
7                               Physical-BMI
8                            Physical-Height
9                            Physical-Weight
10              Physical-Waist_Circumference
11                     Physical-Diastolic_BP
12                        Physical-HeartRate
13                      Physical-Systolic_BP
14                  Fitness_Endurance-Season
15               Fitness_Endurance-Max_Stage
16               Fitness_Endurance-Time_Mins
17                Fitness_Endurance-Time_Sec
18                                FGC-Season
19                                FGC-FGC_CU
20                           FGC-FGC_CU_Zone
21                              FGC-FGC_GSND
22        

In [4]:
smaller_train = train[train.columns.intersection(test.columns)]
#print(smaller_train.columns)

# Crea una lista per memorizzare le righe
rows = []

# Per ogni feature nelle colonne comuni tra train e test
for feature in smaller_train.columns:
    # Calcola la percentuale di valori non nulli nel train e nel test
    train_not_null = 100 * smaller_train[feature].notnull().sum() / len(smaller_train)
    test_not_null = 100 * test[feature].notnull().sum() / len(test)
    
    # Aggiungi il risultato alla lista
    rows.append({'feature': feature, 'train': train_not_null, 'test': test_not_null})

# Converti la lista in un DataFrame
features = pd.DataFrame(rows)
features.sort_values('train', ascending=False, inplace=True)

# Stampa il risultato
print(features)



                                   feature       train   test
0                                       id  100.000000  100.0
1                Basic_Demos-Enroll_Season  100.000000  100.0
2                          Basic_Demos-Age  100.000000  100.0
3                          Basic_Demos-Sex  100.000000  100.0
57                     PreInt_EduHx-Season   89.393939   90.0
18                              FGC-Season   84.494949   85.0
6                          Physical-Season   83.585859   70.0
58  PreInt_EduHx-computerinternet_hoursday   83.358586   80.0
9                          Physical-Weight   77.676768   65.0
8                          Physical-Height   76.439394   65.0
7                             Physical-BMI   76.313131   65.0
12                      Physical-HeartRate   74.924242   60.0
13                    Physical-Systolic_BP   74.595960   55.0
11                   Physical-Diastolic_BP   74.595960   55.0
54                              SDS-Season   66.111111   50.0
55      

## Capiamo come sono fatti i dati
### Non è necessario rendere una pd.Serie le labels del test set. Con la riga sopra eseguita lo fa automaticamente
nella tabella implementata vediamo la percentuale di valori non nulli per ogni feature del dataset sia per il train e per il set: se decidessimo di scartare tutte le colonne che contengono almeno un valore non nullo alleneremo il nostro dataset su 4 features, se decidessimo di allenare il dataset sul 60% dei dati possiamo allenare il nostro modello su 12 features (scartando i dati nulli e le colonne con una percentuale di valori not null non sufficiente)  
sarebbe carino sperimentare il modello su diversi tipi di dati.
Si tenga conto che il test ha solamente 20 istanze quindi avere il 40% di dati not null in una variabile vuoldire avere solamente 8 istanze di test 

In [10]:
smaller_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 59 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      3960 non-null   object 
 1   Basic_Demos-Enroll_Season               3960 non-null   object 
 2   Basic_Demos-Age                         3960 non-null   int64  
 3   Basic_Demos-Sex                         3960 non-null   int64  
 4   CGAS-Season                             2555 non-null   object 
 5   CGAS-CGAS_Score                         2421 non-null   float64
 6   Physical-Season                         3310 non-null   object 
 7   Physical-BMI                            3022 non-null   float64
 8   Physical-Height                         3027 non-null   float64
 9   Physical-Weight                         3076 non-null   float64
 10  Physical-Waist_Circumference            898 non-null    floa

# TO DO:
- Rimuovere le features con la minore Feature Importance
- Rimpiazzare valori NULL con bootstrap sampling dei valori presenti
- Decodificare le Feature Categoriche con 1HE